In [45]:
# disabling contracts for speed
import contracts
import yaml
import numpy as np
import geometry as geo
import numpy as np
from duckietown_world.world_duckietown.tile_template import load_tile_types
from duckietown_world.geo.measurements_utils import iterate_by_class
from duckietown_world.world_duckietown.tile import get_lane_poses
import duckietown_world as dw
from copy import deepcopy
import geometry as g
import pandas as pd

contracts.disable_all()
m = dw.load_map('ethz_amod_lab_k31')


INFO:dt-world:loading map ethz_amod_lab_k31


In [35]:
import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_svg, ipython_draw_html

class Person(dw.PlacedObject):
    def __init__(self, radius, *args, **kwargs):

        self.radius = radius

        dw.PlacedObject.__init__(self, *args, **kwargs)

        

    def draw_svg(self, drawing, g):

        # drawing is done using the library svgwrite

        c = drawing.circle(center=(0, 0), r=self.radius, fill='pink')

        g.add(c)

        # draws x,y axes

        dw.draw_axes(drawing, g)


    def extent_points(self):

        # set of points describing the boundary 

        L = self.radius

        return [(-L, -L), (+L, +L)]

def relative_pose(q0, q1):
    return g.SE2.multiply(g.SE2.inverse(q0), q1)

def interpolate(q0, q1, alpha):
    q1_from_q0 = relative_pose(q0, q1)
    vel = g.SE2.algebra_from_group(q1_from_q0)
    rel = g.SE2.group_from_algebra(vel * alpha)
    q = g.SE2.multiply(q0, rel)
    return q

In [139]:
root = dw.PlacedObject()
import sys
import traceback



realTimestamps = []
with open("autobot29_v3.yaml", 'r') as stream:

    try:

        data = yaml.safe_load(stream)

    except yaml.YAMLError as exc:

        print(exc)

    

    timestart = data['begin_time_stamp']
    data_points = len(data['trajectory_data'])

    x = np.zeros((data_points,))

    y= np.zeros((data_points,))

    R = np.zeros((3,3, data_points))            
    phi = np.zeros((3, data_points))

    

    dx = 999.999*np.ones((data_points, ))

    dy = 999.999*np.ones((data_points, ))

    dr = 999.999*np.ones((data_points, ))
    dphi = 999.999*np.ones((data_points, ))

    final_trajectory = []

    for idx, [time, traj] in enumerate(data['trajectory_data'].items()):

        x[idx] = np.array(traj[0])

        y[idx] = np.array(traj[1])

        R[:,:,idx] = np.reshape(np.array(traj[3:]), (3,3))
        

        phi[:,idx] = np.array([np.arctan2(-R[1,2,idx],R[2,2,idx]), 
                               np.arctan2(R[0,2,idx],np.sqrt(R[0,0,idx]**2 + R[0,1,idx]**2)),
                               np.arctan2(-R[0,1,idx], R[0,0,idx])])

        realTimestamps.append(time)
        #print(phi[:,idx])
        z = phi[2,idx]
        points = np.array([x[idx], y[idx]])
        final_trajectory.append([points, z])
    final_array = final_trajectory
    
seqs2 = []

for entry in range(0, len(final_array)):
    x =  (final_array[entry][0][0] )  # -2.2
    y = final_array[entry][0][1] # + 0.8
    alpha = final_array[entry][1]
    q5 = geo.SE2_from_translation_angle([x,y],alpha)
    
    seqs2.append(q5)

trajectoryFile = 'autobot29_v3.yaml'
path = 'trajectory2Pose/'+str(trajectoryFile.split('.')[0]) + '/image_timestamps.csv'
imagesDFcolumns = ['ImageName', 'Seconds', 'Nanoseconds']
imagesDF = pd.read_csv(path)
timeStampImagesSecondsArray = imagesDF.iloc[:,1]
timeStampImagesNanoSecArray = imagesDF.iloc[:,2]
imageNumber = imagesDF.iloc[:,0]

entryNumberTrajectory = 0
finalArrayWithoutPose = []
finalArrayPoses = []

for entry in range(0, len(timeStampImagesSecondsArray)-1):
    timeStampImagesSeconds = imagesDF.iloc[entry, 1]
    timeStampImagesNanoSec = imagesDF.iloc[entry,2] 
    imageNumber = imagesDF.iloc[entry,0]
    
    # Check if exact time is available in both dataframes
    timeStampImages = str(timeStampImagesSeconds) + '.' + str(timeStampImagesNanoSec)
    timeStampImages = float(timeStampImages)
    timeStampTrajectory = int(timestart) + float(realTimestamps[entryNumberTrajectory])
    timeStampTrajectoryBefore = int(timestart) + float(realTimestamps[entryNumberTrajectory -1]) 
    
    if timeStampImages == timeStampTrajectory:
        q2 = seqs2[entryNumberTrajectory]
        finalArrayPoses.append(q2)
        centerDistance, relativeHeading = calculatePose(q2)
        finalArrayWithoutPose.append([imageNumber, timeStampImages, centerDistance, relativeHeading])
        entryNumberTrajectory += 1
        
    elif timeStampImages < timeStampTrajectory:
        timeStamp2 = timeStampTrajectory
        timeStampWanted = timeStampImages
        timeStamp1 = timeStampTrajectoryBefore
        param = timeStampWanted / (timeStamp2 - timeStamp1)
        
        q2 = seqs2[entryNumberTrajectory]
        q1 = seqs2[entryNumberTrajectory-1]

        qInter = interpolate(q1, q2, param)
        finalArrayPoses.append(qInter)
        centerDistance, relativeHeading = calculatePose(qInter)
        finalArrayWithoutPose.append([imageNumber, timeStampImages, centerDistance, relativeHeading])
    
    
    elif timeStampImages > timeStampTrajectory:
        while timeStampImages > timeStampTrajectory:
            entryNumberTrajectory += 1
            lenghtTrajectoryArray = len(realTimestamps)
            if entryNumberTrajectory > lenghtTrajectoryArray:
                break
            
            timeStampTrajectory = int(timestart) + float(realTimestamps[entryNumberTrajectory])
            timeStampTrajectoryBefore = int(timestart) + float(realTimestamps[entryNumberTrajectory -1]) 
        if timeStampImages < timeStampTrajectory:
            timeStamp2 = timeStampTrajectory
            timeStampWanted = timeStampImages
            timeStamp1 = timeStampTrajectoryBefore
            param = timeStampWanted / (timeStamp2 - timeStamp1)

            q2 = seqs2[entryNumberTrajectory]
            q1 = seqs2[entryNumberTrajectory-1]

            qInter = interpolate(q1, q2, param)
            finalArrayPoses.append(qInter)
            centerDistance, relativeHeading = calculatePose(qInter)
            finalArrayWithoutPose.append([imageNumber, timeStampImages, centerDistance, relativeHeading])


    else:
        print('unhandled')
finalArrayWithoutPose = np.array(finalArrayWithoutPose)
print(finalArrayWithoutPose)

[[8.60000000e+01 1.57348902e+09 1.62058084e-01 1.90198164e-01]
 [8.70000000e+01 1.57348902e+09 1.62364576e-01 1.90342359e-01]
 [8.80000000e+01 1.57348902e+09 1.62647185e-01 1.90476116e-01]
 ...
 [1.04300000e+03 1.57348905e+09 1.00000000e+01 0.00000000e+00]
 [1.04400000e+03 1.57348905e+09 1.00000000e+01 0.00000000e+00]
 [1.04500000e+03 1.57348905e+09 1.00000000e+01 0.00000000e+00]]


In [130]:
def calculatePose(qPose):
    timestamps = 1 # [0, 1, 2, ...]
    # SE2Transform is the wrapper for SE2 used by Duckietown World 
    transforms = dw.SE2Transform.from_SE2(qPose)
    
    #transforms = [dw.SE2Transform.from_SE2(_) for _ in qPose]
    #seq_me = dw.SampledSequence(timestamps, transforms)
    counter = 0
    center_points2 = []

    timestamp = 1
    pose_object = transforms
    hoi = list(get_lane_poses(m, pose_object.as_SE2()))
    if len(hoi) == 0:
        return 10, 0
        #center_points2.append(dw.SE2Transform.from_SE2(geo.SE2_from_translation_angle([0,0],0)))

    else:
        #print('outside left: ' + str(hoi[0].lane_pose.distance_from_left) + ' outside right: ' + str(hoi[0].lane_pose.distance_from_right))
        distance_from_left = hoi[0].lane_pose.distance_from_left
        distance_from_right = hoi[0].lane_pose.distance_from_right
        distance_from_center = hoi[0].lane_pose.distance_from_center
        rel_heading = hoi[0].lane_pose.relative_heading

        #absTime = float(relTimestamps[counter]) + int(timestart)
        return distance_from_center, rel_heading
        #center_points2.append(hoi[0].lane_pose.center_point)
        #print(hoi[0].lane_pose.distance_from_center, hoi[0].lane_pose.relative_heading)
    #lane_pose = lanesegment.lane_pose_from_SE2Transform(pose_object)    
    #center_points.append(lane_pose.center_point)
    #print(relTimestamps[counter])

    counter += 1

    print(counter)









In [137]:
# we now add a marker for projection in the center point
#sequence = dw.SampledSequence(seq_me.timestamps, seq_me.center_points)
#m.set_object("projection2", dw.PlacedObject(), ground_truth=seq_me)
m = dw.load_map('ethz_amod_lab_k31')

m.set_object("me", Person(0.1), ground_truth=seq_me)


INFO:dt-world:loading map ethz_amod_lab_k31


In [138]:
ipython_draw_html(m);



INFO:dt-world:area: RectangularArea(pmin=[0. 0.],pmax=[3.45617996 3.45617996])
INFO:dt-world:Written SVG to out/ipython_draw_html/140018358189008/drawing.svg
INFO:dt-world:Written HTML to out/ipython_draw_html/140018358189008/drawing.html


In [136]:

timestamps = range(len(finalArrayPoses)) # [0, 1, 2, ...]
print(len(timestamps))
# SE2Transform is the wrapper for SE2 used by Duckietown World 
transforms = [dw.SE2Transform.from_SE2(_) for _ in finalArrayPoses]
print(len(transforms))
seq_me = dw.SampledSequence(timestamps, transforms)
print(seq_me)
counter = 0
center_points2 = []

for timestamp, pose_object in seq_me:
    break

    hoi = list(get_lane_poses(m, pose_object.as_SE2()))
    if len(hoi) == 0:
        print(dw.SE2Transform.from_SE2(geo.SE2_from_translation_angle([0,0],0)))
        #center_points2.append(dw.SE2Transform.from_SE2(geo.SE2_from_translation_angle([0,0],0)))
        
    else:
        #print('outside left: ' + str(hoi[0].lane_pose.distance_from_left) + ' outside right: ' + str(hoi[0].lane_pose.distance_from_right))
        distance_from_left = hoi[0].lane_pose.distance_from_left
        distance_from_right = hoi[0].lane_pose.distance_from_right
        distance_from_center = hoi[0].lane_pose.distance_from_center
        rel_heading = hoi[0].lane_pose.relative_heading
        print(timestamp, distance_from_center, rel_heading) 
        
        #absTime = float(relTimestamps[counter]) + int(timestart)
        center_points2.append(hoi[0].lane_pose.center_point)
        #print(hoi[0].lane_pose.distance_from_center, hoi[0].lane_pose.relative_heading)
    #lane_pose = lanesegment.lane_pose_from_SE2Transform(pose_object)    
    #center_points.append(lane_pose.center_point)
    #print(relTimestamps[counter])

    counter += 1

    print(counter)

412
412
SampledSequence(timestamps=[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153